In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## 1. Training Network

In [4]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.nn.modules import Module
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import sys
model_dir = '/homes/okz21/NNC/models/'
sys.path.append(model_dir)
import models 


#Data
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='./data', train=False , transform=transforms.ToTensor(), download=True)

In [3]:
reload(models)
model = models.SWSModel()
for params in model.parameters():
    print params.size()
    
num_epochs = 100
batch_size = 256

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
use_cuda = torch.cuda.is_available()
print use_cuda

torch.Size([25, 1, 5, 5])
torch.Size([25])
torch.Size([50, 25, 3, 3])
torch.Size([50])
torch.Size([500, 1250])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])
True


In [15]:
def test_accuracy(test_loader,model):
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Iterate through test dataset
    for images, labels in test_loader:
        if use_cuda:
            images=images.cuda()
        images = Variable(images)
        # Forward pass only to get logits/output
        outputs = model(images)
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
        # Total number of labels
        total += labels.size(0)
        if use_cuda:
            correct += (predicted.cpu() == labels.cpu()).sum()
        else:
            correct += (predicted == labels).sum()

    accuracy = 100.0 * correct / total
    return accuracy
    
def train_epoch(model, optimizer, criterion, train_loader):
    for i, (images, labels) in enumerate(train_loader):

        if(use_cuda):
            images=images.cuda()
            labels=labels.cuda()
        images = Variable(images)
        labels = Variable(labels)
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        # Forward pass to get output/logits
        outputs = model(images)
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        # Getting gradients w.r.t. parameters
        loss.backward()
        # Updating parameters
        optimizer.step()
    return model, loss

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

if use_cuda:
    model.cuda()
for epoch in range(num_epochs):
    model, loss = train_epoch(model, optimizer, criterion, train_loader)
    accuracy = test_accuracy(test_loader, model)
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.data[0], accuracy))

RuntimeError: torch.FloatTensor constructor doesn't accept any keyword arguments

In [31]:
torch.save(model, model_dir + 'mnist_{}_{}.m'.format(model.name, num_epochs))

## 2. Re-training with empirical prior

In [126]:
x=100000
t = torch.Tensor(x,x)
t.random_()

KeyboardInterrupt: 

In [125]:
def logsumexp(t, axis=1):
    t_max, _ = t.max(dim=axis)
    t = t-t_max.repeat(t_max.size(1), t_max.size(0))
    t.exp_()
    t = t.sum(dim=axis)
    t.log_()
    return t + t_max

t = torch.Tensor([[1, 2, 3, 4] , [1, 2, 3, 4], [100, 200, 300, 400], [100, 200, 300, 400]])
print logsumexp(t, axis=0)


 100.6931  200.6931  300.6931  400.6931
[torch.FloatTensor of size 1x4]



In [ ]:
def logsumexp(t, w=None, axis=1):
    """
    t: tensor
    w: weight tensor
    axis: matrix-axis to find max and sum along
    """
    if w is not None:
        



In [5]:
model = model=torch.load(model_dir + 'mnist_{}_{}.m'.format(model.name, num_epochs))

In [25]:
#Build
J = 16
pi_zero=0.99
# create trainable ...
#    ... means
init_mean = Variable(torch.linspace(-0.6, 0.6, J - 1))

init_stds = Variable(torch.FloatTensor(np.tile(0.25, J) ))
init_gamma = - torch.log(torch.pow(init_stds, 2))


init_mixing_proportions = torch.ones((J - 1))
init_mixing_proportions *= (1. - pi_zero) / (J - 1)

trainable_weights = [init_mean] + [init_gamma] + [init_mixing_proportions]
trainable_weights
#[p.view(-1) for p in model.parameters()]


[Variable containing:
 -0.6000
 -0.5143
 -0.4286
 -0.3429
 -0.2571
 -0.1714
 -0.0857
  0.0000
  0.0857
  0.1714
  0.2571
  0.3429
  0.4286
  0.5143
  0.6000
 [torch.FloatTensor of size 15], Variable containing:
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
  2.7726
 [torch.FloatTensor of size 16], 
 1.00000e-04 *
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
   6.6667
 [torch.FloatTensor of size 15]]

In [ ]:
def call(self, x, mask=None):
    J = self.nb_components
    loss = K.variable(0.)
    # here we stack together the trainable and non-trainable params
    #     ... the mean vector
    means = K.concatenate([K.variable([0.]), self.means],axis=0)
    #     ... the variances
    precision = K.exp(self.gammas)
    #     ... the mixing proportions (we are using the log-sum-exp trick here)
    min_rho = K.min(self.rhos)
    mixing_proportions = K.exp(self.rhos - min_rho)
    mixing_proportions = (1 - self.pi_zero) * mixing_proportions / K.sum(mixing_proportions)
    mixing_proportions = K.concatenate([K.variable([self.pi_zero]), mixing_proportions],axis=0)

    # compute the loss given by the gaussian mixture
    for weights in self.network_weights:
        loss = loss + self.compute_loss(weights, mixing_proportions, means, precision)

    # GAMMA PRIOR ON PRECISION
    # ... for the zero component
    (alpha, beta) = (5e3,20e-1)
    neglogprop = (1 - alpha) * K.gather(self.gammas, [0]) + beta * K.gather(precision, [0])
    loss = loss + K.sum(neglogprop)
    # ... and all other components
    alpha, beta = (2.5e2,1e-1)
    idx = np.arange(1, J)
    neglogprop = (1 - alpha) * K.gather(self.gammas, idx) + beta * K.gather(precision, idx)
    loss = loss + K.sum(neglogprop)

    return loss

In [40]:
class GaussianMixturePrior(Module):
    def __init__(self, nb_components, network_weights, pretrained_weights, pi_zero, **kwargs):
        super(GaussianMixturePrior, self).__init__()
        self.nb_components = nb_components 
        #self.network_weights = [K.flatten(w) for w in network_weights]
        self.network_weights = [p.view(-1) for p in network_weights]
        #self.pretrained_weights = special_flatten(pretrained_weights)
        self.pretrained_weights = [p.view(-1) for p in pretrained_weights]
        self.pi_zero = pi_zero
        
        #Build
        J = self.nb_components
        pi_zero = self.pi_zero
        # create trainable ...
        #    ... means
        init_mean = torch.linspace(-0.6, 0.6, J - 1)
        self.mean = Variable(init_mean)
        
        init_stds = torch.FloatTensor(np.tile(0.25, J) )
        self.gammas = Variable(- torch.log(torch.pow(init_stds, 2)))
        

        init_mixing_proportions = torch.ones((J - 1))
        init_mixing_proportions *= (1. - pi_zero) / (J - 1)
        self.rhos = Variable(init_mixing_proportions)

        self.trainable_weights = [init_mean] + [init_gamma] + [init_mixing_proportions]
        
        #Call
        loss = Variable(torch.FloatTensor([0.]))
        means = torch.cat(( Variable(torch.FloatTensor([0.])) , init_mean), 0)
        precision = torch.exp(init_gamma)


gmp = GaussianMixturePrior(16, [x for x in model.parameters()], [x for x in model.parameters()], 0.99)

[
-0.6000
-0.5143
-0.4286
-0.3429
-0.2571
-0.1714
-0.0857
 0.0000
 0.0857
 0.1714
 0.2571
 0.3429
 0.4286
 0.5143
 0.6000
[torch.FloatTensor of size 15]
, Variable containing:
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
 2.7726
[torch.FloatTensor of size 16]
, 
1.00000e-04 *
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
  6.6667
[torch.FloatTensor of size 15]
]
